# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096840


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:00,  2.18s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:02,  2.31s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:39,  1.63s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:28,  1.22s/it]

...

Rendering models:  77%|███████▋  | 23/30 [00:09<00:00,  7.11it/s]

Rendering models:  80%|████████  | 24/30 [00:09<00:00,  6.55it/s]

Rendering models:  83%|████████▎ | 25/30 [00:10<00:00,  5.96it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  7.83it/s]

whitgift                              0.001753
Jnursssmith5263                       0.001443
cameronw                              0.013354
Jonas_Cross                           0.000394
jnarayanbvg                           0.010084
TateK                                 0.003983
not-logged-in-c6917d9b19f41e6c2995    0.002232
awright5                              0.003590
thiagoxxxx                            0.000330
not-logged-in-4ec54f98b3c226b9b959    0.001908
clars915                              0.000473
coyotes54                             0.000356
BraxtonKinney                         0.000562
not-logged-in-300dca7f45d1a5d1dd58    0.021052
TRISTANM18                            0.000237
stansall                              0.000273
Lavadude                              0.057217
not-logged-in-88d5e424af552d0f2955    0.005369
LeusaneLordelo                        0.000861
IdeasInSpace                          0.000396
not-logged-in-fded2070a0a9c0a5b66f    0.000950
FR4N1O       

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())